# The Travelling Salesman Problem

## Description

The Travelling Salesman Problem (TSP) is a classic algorithmic problem in the fields of computer science and operations research. It focuses on optimization. In this problem, a salesman is given a list of cities and must determine the shortest possible route that visits each city exactly once and returns to the origin city. The challenge is to find the most efficient route among the possible permutations of cities. TSP has applications in logistics, planning, and the manufacturing of microchips. It is an NP-hard problem, meaning that there is no known efficient way to solve it for large numbers of cities.

## Definition

### General Model

Given a set of cities and the distances between each pair of cities, the goal is to find the shortest possible tour that visits each city exactly once and returns to the starting city.

Mathematically, the problem can be formulated as follows:

- Let $C$ be a set of $n$ cities.
- Let $d(i, j)$ be the distance between city $i$ and city $j$.

The objective is to find a permutation $\pi$ of $C$ that minimizes the total tour length:

$$ \text{Minimize} \sum_{i=1}^{n-1} d(\pi(i), \pi(i+1)) + d(\pi(n), \pi(1)) $$

where $\pi(i)$ represents the $i$-th city in the permutation.

### Model based on Graph Representation

The Travelling Salesman Problem can also be represented using graph theory. In this representation:

- Each city is represented as a vertex.
- Each pair of cities is connected by an edge.
- The weight of an edge represents the distance between the two cities.

Formally, let $G = (V, E)$ be a complete graph where:
- $V$ is the set of vertices representing the cities.
- $E$ is the set of edges representing the paths between the cities.
- $w: E \rightarrow \mathbb{R}^+$ is a weight function that assigns a positive real number to each edge, representing the distance between the cities.

The objective is to find a Hamiltonian cycle (a cycle that visits each vertex exactly once and returns to the starting vertex) with the minimum total weight.

Mathematically, the problem can be formulated as:

$$ \text{Minimize} \sum_{(u,v) \in E} w(u,v) \cdot x_{uv} $$

subject to:

1. Each vertex is visited exactly once:
    $$ \sum_{v \in V} x_{uv} = 1 \quad \forall u \in V $$
    $$ \sum_{u \in V} x_{uv} = 1 \quad \forall v \in V $$

2. Subtour elimination constraints to ensure a single tour:
    $$ \sum_{(u,v) \in S} x_{uv} \leq |S| - 1 \quad \forall S \subset V, 2 \leq |S| \leq |V| - 1 $$

3. Nature of the variables:
    $$ x_{uv} \in \{0, 1\} \quad \forall (u,v) \in E $$

where $x_{uv}$ is a binary variable that is 1 if the edge $(u,v)$ is included in the tour and 0 otherwise.

## A Word on Subtour Elimination Constraints

Subtour elimination constraints are necessary to ensure that the solution to the Travelling Salesman Problem (TSP) forms a single tour that visits each city exactly once and returns to the starting city. Without these constraints, the solution might consist of multiple disconnected subtours, which do not satisfy the requirements of the TSP.

### Why Subtour Elimination Constraints are Needed

In the context of the TSP, a subtour is a smaller tour that visits a subset of cities but does not include all cities or return to the starting city. Subtour elimination constraints prevent these smaller tours from forming by ensuring that any subset of cities does not form a closed loop unless it includes all cities.

### Alternatives to Subtour Elimination Constraints

1. **Miller-Tucker-Zemlin (MTZ) Constraints**: These constraints introduce additional variables to keep track of the order in which cities are visited. They are simpler to implement but can be less efficient for larger instances of the TSP.

2. **Flow-based Formulations**: These formulations use flow variables to model the movement between cities, ensuring that the flow is consistent with a single tour. This approach can be more complex but may provide tighter formulations.

3. **Cutting Plane Methods**: These methods iteratively add subtour elimination constraints as needed during the solution process. This approach can be more efficient as it avoids adding all possible constraints upfront.

Each of these alternatives has its own advantages and trade-offs in terms of complexity, computational efficiency, and ease of implementation.


## An Illustrative Example

We will explore an example of the Travelling Salesman Problem (TSP) using a dataset stored in "assets/TSP.csv". This CSV file contains the following columns:

- `City`: The name of the city.
- `X`: The x-coordinate of the city.
- `Y`: The y-coordinate of the city.

These coordinates represent the positions of the cities on a 2D plane, and the goal is to find the shortest possible route that visits each city exactly once and returns to the origin city.

We will use a graph representation to model this problem

### Setiing the Environment

In [ ]:
%pip install pulp matplotlib pandas

import pandas as pd
import matplotlib.pyplot as plt
import pulp as pl